## Remove a subject from the data (patient R018)

In [43]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import pickle
import mat73

In [2]:
# Get data path
cwd = Path(os.getcwd())
pardir = str(cwd.parent.absolute())
projdir = os.path.dirname(pardir)
projdir = os.path.dirname(projdir)
pardir, projdir

('/home/alessio/projects/Blindsight_LEIDA', '/home/alessio')

In [37]:
# load the subject list and find the position of the subject I want to remove
classif = pd.read_excel(pardir+'/data/MD_05122023/davide_class_blindsight.xlsx')
sub_idxs = classif['ID'].values
my_sub = 'R018'
my_sub_idx = np.where(sub_idxs==my_sub)[0][0]

In [38]:
classif_noR018 = classif.drop(index=my_sub_idx)
classif_noR018.to_excel(pardir+'/data/MD_05122023/davide_class_blindsight_noR018.xlsx')

In [49]:
# Vector indicating subject at each time points
path =pardir + '/data/MD_05122023/time_all.pkl'
with open(path, 'rb') as pickle_file:
    time_all = pickle.load(pickle_file)
T = time_all.shape[1]/33 # time points of a single subject
# I havve to cut the time points related to subject I want to remove
print(time_all.shape)
time_all_noR018 = np.concatenate((time_all[:,0:int(my_sub_idx*T)], time_all[:,int((my_sub_idx+1)*T):]), axis=1)
print(time_all_noR018.shape)
with open(pardir + '/data/MD_05122023_noR018/time_all_noR018.pkl', 'wb') as handle:
    pickle.dump(time_all_noR018, handle, protocol=pickle.HIGHEST_PROTOCOL)

(1, 14223)
(1, 13792)


In [46]:
# Load full matrix
path =pardir + '/data/MD_05122023/PL.mat'
PL = mat73.loadmat(path)
plm_tc = np.transpose(PL['PL_all_alessio'], (2,0,1))
print(plm_tc.shape)

# Remove the missing subject from the concatenated phase locking matrices
plm_tc_noR018 = np.concatenate((plm_tc[0:int(my_sub_idx*T)], plm_tc[int((my_sub_idx+1)*T):]), axis=0)
print(plm_tc_noR018.shape)
with open(pardir + '/data/MD_05122023_noR018/plm_tc_noR018.pkl', 'wb') as handle:
    pickle.dump(plm_tc_noR018, handle, protocol=pickle.HIGHEST_PROTOCOL)

(14223, 234, 234)


In [51]:
# # Load eigenvectors decomposition
path = pardir + '/data/MD_05122023/data_V1.mat'
Ev_ = mat73.loadmat(path)
Ev = Ev_['data']
print(Ev.shape)

# Remove the missing subject from the concatenated eigenvectors
Ev_noR018 = np.concatenate((Ev[0:int(my_sub_idx*T)], Ev[int((my_sub_idx+1)*T):]), axis=0)
print(Ev_noR018.shape)
with open(pardir + '/data/MD_05122023_noR018/Ev_noR018.pkl', 'wb') as handle:
    pickle.dump(Ev_noR018, handle, protocol=pickle.HIGHEST_PROTOCOL)

(14223, 234)
(13792, 234)


In [52]:
# Load K-means results
# state_labels
path = pardir + '/data/MD_05122023/clustering_res.mat'
kmeans_leida_ = mat73.loadmat(path)
print(kmeans_leida_.keys())
kmeans_leida = kmeans_leida_['Kmeans_results']
# kmeans_leida = kmeans_leida_['Kmeans_results']
K_max = len(kmeans_leida) + 3
print(kmeans_leida[0].keys())

dict_keys(['Kmeans_results', 'fuzzy_results', 'fuzzy_results_clean', 'mal_dist_kmean'])
dict_keys(['C', 'D', 'IDX', 'SUMD'])


In [55]:
kmeans_leida

[{'C': array([[-0.0652061 , -0.06007689, -0.06697078, -0.06715395, -0.0643249 ,
          -0.03761628, -0.0664506 , -0.06674304, -0.06682051, -0.06879738,
          -0.06657563, -0.06315919, -0.06651627, -0.06801122, -0.06760977,
          -0.05652805, -0.06733161, -0.06664699, -0.06813493, -0.06920203,
          -0.06973635, -0.06851936, -0.06632408, -0.07040984, -0.06838866,
          -0.06867668, -0.06795616, -0.06868483, -0.0693437 , -0.06863892,
          -0.06395263, -0.06605231, -0.06814228, -0.06588752, -0.06549974,
          -0.06845804, -0.06966876, -0.06220425, -0.06594389, -0.06682819,
          -0.0687555 , -0.06792215, -0.06914359, -0.0698    , -0.06939982,
          -0.07049859, -0.06818376, -0.06356914, -0.07048581, -0.06890887,
          -0.06668189, -0.06269897, -0.06509036, -0.06682471, -0.06699559,
          -0.06849914, -0.06872843, -0.06901555, -0.06718153, -0.06833683,
          -0.0699215 , -0.0690061 , -0.06851828, -0.07067693, -0.05777926,
          -0.0690892

In [56]:
# Keys: IDX: state index for each time point, C: centroids coordinates, SUMD: buh, D: buh
k = 4
state_labels = kmeans_leida[k]['IDX']
centroids = kmeans_leida[k]['C']

In [58]:
state_labels.shape

(14223,)